In [1]:
import numpy as np

class NeuralNetworkWithMomentum:
    def __init__(self, learning_rate=0.005, momentum=0.9):
        # Paramètres d'apprentissage
        self.learning_rate = learning_rate
        self.momentum = momentum
        
        # Initialisation des poids et biais
        self.W1 = np.random.randn(2, 4) * 0.01  # 2 entrées, 4 neurones
        self.b1 = np.zeros((1, 4))
        self.W2 = np.random.randn(4, 1) * 0.01  # 4 neurones, 1 sortie
        self.b2 = np.zeros((1, 1))
        
        # Initialisation des vélocités
        self.vW1 = np.zeros_like(self.W1)
        self.vb1 = np.zeros_like(self.b1)
        self.vW2 = np.zeros_like(self.W2)
        self.vb2 = np.zeros_like(self.b2)

    def relu(self, Z):
        """Fonction d'activation ReLU"""
        return np.maximum(0, Z)
    
    def relu_derivative(self, Z):
        """Dérivée de ReLU"""
        return Z > 0
    
    def sigmoid(self, Z):
        """Fonction d'activation Sigmoid"""
        return 1 / (1 + np.exp(-Z))
    
    def forward_propagation(self, X):
        """Propagation avant"""
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = self.relu(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = self.sigmoid(self.Z2)
        return self.A2
    
    def compute_loss(self, y, y_pred):
        """Calcul de la perte (MSE)"""
        return np.mean((y - y_pred) ** 2)
    
    def backward_propagation_with_momentum(self, X, y):
        """Rétropropagation avec momentum"""
        m = X.shape[0]
        
        # Gradients de la couche de sortie
        dZ2 = self.A2 - y
        dW2 = np.dot(self.A1.T, dZ2) / m
        db2 = np.mean(dZ2, axis=0)
        
        # Gradients de la première couche
        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * self.relu_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / m
        db1 = np.mean(dZ1, axis=0)
        
        # Mise à jour des vélocités
        self.vW1 = self.momentum * self.vW1 - self.learning_rate * dW1
        self.vb1 = self.momentum * self.vb1 - self.learning_rate * db1
        self.vW2 = self.momentum * self.vW2 - self.learning_rate * dW2
        self.vb2 = self.momentum * self.vb2 - self.learning_rate * db2
        
        # Mise à jour des poids avec momentum
        self.W1 += self.vW1
        self.b1 += self.vb1
        self.W2 += self.vW2
        self.b2 += self.vb2

    def train(self, X, y, epochs=1000):
        """Entraînement du réseau"""
        losses = []
        
        for epoch in range(epochs):
            # Propagation avant
            y_pred = self.forward_propagation(X)
            
            # Calcul de la perte
            loss = self.compute_loss(y, y_pred)
            losses.append(loss)
            
            # Rétropropagation avec momentum
            self.backward_propagation_with_momentum(X, y)
            
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")
        
        return losses

# Test du réseau avec et sans momentum
if __name__ == "__main__":
    # Génération de données synthétiques
    np.random.seed(42)
    X = np.random.randn(100, 2)
    y = (X[:, 0] + X[:, 1] > 0).reshape(-1, 1).astype(float)

    # Test avec momentum
    print("Entraînement avec momentum (coefficient = 0.9):")
    model_with_momentum = NeuralNetworkWithMomentum(momentum=0.9)
    losses_with_momentum = model_with_momentum.train(X, y)

    # Test sans momentum
    print("\nEntraînement sans momentum (coefficient = 0):")
    model_without_momentum = NeuralNetworkWithMomentum(momentum=0)
    losses_without_momentum = model_without_momentum.train(X, y)

    # Comparaison des résultats
    print("\nComparaison des pertes finales:")
    print(f"Avec momentum: {losses_with_momentum[-1]:.4f}")
    print(f"Sans momentum: {losses_without_momentum[-1]:.4f}")


Entraînement avec momentum (coefficient = 0.9):
Epoch 0, Loss: 0.2500
Epoch 100, Loss: 0.2463
Epoch 200, Loss: 0.2425
Epoch 300, Loss: 0.2100
Epoch 400, Loss: 0.1259
Epoch 500, Loss: 0.0746
Epoch 600, Loss: 0.0490
Epoch 700, Loss: 0.0351
Epoch 800, Loss: 0.0270
Epoch 900, Loss: 0.0222

Entraînement sans momentum (coefficient = 0):
Epoch 0, Loss: 0.2500
Epoch 100, Loss: 0.2492
Epoch 200, Loss: 0.2486
Epoch 300, Loss: 0.2481
Epoch 400, Loss: 0.2477
Epoch 500, Loss: 0.2474
Epoch 600, Loss: 0.2471
Epoch 700, Loss: 0.2469
Epoch 800, Loss: 0.2468
Epoch 900, Loss: 0.2466

Comparaison des pertes finales:
Avec momentum: 0.0191
Sans momentum: 0.2465
